# Init

In [3]:
import glob
# import matplotlib as mpl
# import matplotlib.cm as cm
import numpy as np
import os
import PIL.Image as pil
import torch

from shutil import copyfile
from tqdm import tqdm
from torchvision import transforms, datasets

# to import the pretrained model, first change working dir to its path,
# then change back
os.chdir('/home/yu/OneDrive/Construal/pretrained_MonoDepth2')

import models, networks
from layers import disp_to_depth

os.chdir('/home/yu/OneDrive/Construal')


# Resave image data

> Create a new folder `data/Kickstarter Image/pid` which stores profile images for project with id `pid`. 

In [ ]:
def find_files(filename, search_path='data/Kickstarter Data'):
    image_fullnames = []

    # Walking top-down from the root
    print('Searing for `profile_full.jpg`...')
    for i, (root, dir, files) in enumerate(os.walk(search_path)):
      # if i > 1e0:
      #    break

        if filename in files:
            image_fullnames.append(os.path.join(root, filename).replace('\\', '/'))

        assert len(set(image_fullnames)) == len(image_fullnames), 'Duplicate `profile_full.jpg` detected!'

    # copy images to `data/Kictstarter Image`
    print('Saving images...')
    for name in tqdm(image_fullnames):
        # get pid (project id)
        pid = name.split('/')[-2]

        # create one folder for each project, if not exist
        pdir = f'data/KickStarter Image/{pid}'
        if not os.path.exists(pdir):
            os.mkdir(pdir)

        copyfile(name, f'{pdir}/profile_full.jpg')

    print('Done!')
    return image_fullnames

profile_images = find_files("profile_full.jpg")

# Estismate Depth

## Model parameters

In [3]:
# list of image paths
paths = glob.glob('data/Kickstarter Image/*/profile_full.jpg')

# model path
model_name = 'mono+stereo_640x192'
model_path = f'pretrained_MonoDepth2/models/{model_name}'
device = torch.device("cuda")

## Load models

In [9]:
print("-> Loading model from ", model_path)
encoder_path = f'{model_path}/encoder.pth'
depth_decoder_path = f'{model_path}/depth.pth'

# LOADING PRETRAINED MODEL
print("   Loading pretrained encoder")
encoder = networks.ResnetEncoder(18, False)
loaded_dict_enc = torch.load(encoder_path, map_location=device)

# extract the height and width of image that this model was trained with

feed_height = loaded_dict_enc['height']
feed_width = loaded_dict_enc['width']
filtered_dict_enc = {k: v for k, v in loaded_dict_enc.items() if k in encoder.state_dict()}
encoder.load_state_dict(filtered_dict_enc)
encoder.to(device)
encoder.eval()

print("   Loading pretrained decoder")
depth_decoder = networks.DepthDecoder(
    num_ch_enc=encoder.num_ch_enc, scales=range(4))

loaded_dict = torch.load(depth_decoder_path, map_location=device)
depth_decoder.load_state_dict(loaded_dict)

depth_decoder.to(device)
depth_decoder.eval();

-> Loading model from  pretrained_MonoDepth2/models/mono+stereo_640x192
   Loading pretrained encoder
   Loading pretrained decoder


## Predict

In [ ]:
# FINDING INPUT IMAGES
print("-> Predicting on {:d} test images".format(len(paths)))

# PREDICTING ON EACH IMAGE IN TURN
with torch.no_grad():
    for idx, image_path in enumerate(tqdm(paths)):
        image_path = image_path.replace('\\','/')

        # don't try to predict disparity for a disparity image!
        if image_path.endswith("_disp.jpg"):
            continue

        # if the img is already processed, skip
        image_name = os.path.splitext(os.path.basename(image_path))[0]
        pfolder = os.path.dirname(image_path)
        name_dest_npy = f'{pfolder}/{image_name}_md2.npy'
        name_dest_im = f'{pfolder}/{image_name}_md2.jpg'

        if os.path.exists(name_dest_im) and os.path.exists(name_dest_npy):
            continue

        try:
            # Load image and preprocess
            input_image = pil.open(image_path).convert('RGB')
            original_width, original_height = input_image.size
            input_image = input_image.resize((feed_width, feed_height), pil.LANCZOS)
            input_image = transforms.ToTensor()(input_image).unsqueeze(0)

            # PREDICTION
            input_image = input_image.to(device)
            features = encoder(input_image)
            outputs = depth_decoder(features)

            disp = outputs[("disp", 0)]
            disp_resized = torch.nn.functional.interpolate(
                disp, (original_height, original_width), mode="bilinear", align_corners=False)

            # Saving numpy file
            scaled_disp, _ = disp_to_depth(disp, 0.1, 100)
            np.save(name_dest_npy, scaled_disp.cpu().numpy())

            # Saving colormapped depth image
            disp_resized_np = disp_resized.squeeze().cpu().numpy()
            vmax = np.percentile(disp_resized_np, 95)
            normalizer = mpl.colors.Normalize(vmin=disp_resized_np.min(), vmax=vmax)
            mapper = cm.ScalarMappable(norm=normalizer, cmap='magma')
            colormapped_im = (mapper.to_rgba(disp_resized_np)[:, :, :3] * 255).astype(np.uint8)
            im = pil.fromarray(colormapped_im)
            im.save(name_dest_im)

        except Exception as e:
            print(f'Exception @ {image_path}: {e}')
            


print('-> Done!')

In [ ]:
import numpy as np

x = np.load('/home/yu/OneDrive/Construal/data/Kickstarter Image/1750261/profile_full_md2.npy')
x[0,0,0,...]

In [5]:
x.shape

(1, 1, 192, 640)